In [1]:
import requests
from bs4 import BeautifulSoup as BS, element as BS_element
import lxml
import pandas as pd

In [2]:
rutor_url = 'http://5.45.84.133/new/'
host = '/'.join(rutor_url.split('/', maxsplit=3)[:3])
response = requests.get(rutor_url)
response.status_code

200

In [3]:
soup = BS(response.content, 'lxml')
main_table = soup.find('div', id='index')
dependent_tables: tuple[BS_element.Tag] = tuple(main_table.find('h2').find_next_siblings())

In [4]:
def _get_from_table(table: BS_element.Tag, group_name: str) -> pd.DataFrame:
    elements: tuple[BS_element.Tag] = tuple(table.find_all('td'))
    BS_data, BS_name, BS_size, BS_pirs = elements[0],elements[1],elements[-2],elements[-1]
    
    data = BS_data.get_text(strip=True)
    
    BS_name: BS_element.Tag = tuple(BS_name.find_all('a'))[-1]
    name = BS_name.get_text(strip=True)
    href = host + BS_name.get('href')

    size = BS_size.get_text(strip=True)
    
    up, down = tuple(map(int, BS_pirs.stripped_strings))
    
    return pd.DataFrame({
        'group': group_name,
        'data': data,
        'name': name,
        'size': size,
        'up': up,
        'down': down,
        'href': href
    }, index=[0])
    
    

def _get_table(elements: tuple[BS_element.Tag, BS_element.Tag]) -> list[pd.DataFrame]:
    bs_name, bs_table = elements
    group_name = bs_name.find('a').get_text(strip=True)
    group_tables: tuple[BS_element.Tag] = tuple(bs_table.find('tr', class_='backgr').find_next_siblings('tr'))
    
    ap_dataframe = []
    for item in group_tables:
        ap_dataframe.append(_get_from_table(item, group_name))
    
    return ap_dataframe
    
    
rearranged_tables: tuple[tuple[BS_element.Tag, BS_element.Tag]] = tuple(zip(dependent_tables[0::2], dependent_tables[1::2]))
ap_dataframe = []
for item in rearranged_tables:
    ap_dataframe.append(_get_table(item))

ap_dataframe = sum(ap_dataframe, [])
df = pd.concat(ap_dataframe, ignore_index=True, sort= False)

In [5]:
test = df['group'].unique()
print(test)
interesting = test[[0, 3, 6, 7, 10]]

['Зарубежные фильмы' 'Наши фильмы' 'Научно-популярные фильмы'
 'Зарубежные сериалы' 'Наши сериалы' 'Телевизор' 'Мультипликация' 'Аниме'
 'Иностранные релизы' 'Музыка' 'Игры' 'Софт' 'Спорт и Здоровье' 'Юмор'
 'Хозяйство и Быт' 'Книги' 'Другое']


In [6]:
my_df = df[df['group'].isin(interesting)]

cut = 40
my_df = my_df[my_df['up'] >= cut]

display(my_df[my_df.columns[:-1]])

,group,data,name,size,up,down
12,Зарубежные фильмы,08 Май 23,Астерикс и Обеликс: Поднебесная / Astérix & Ob...,745.46 MB,46,17
13,Зарубежные фильмы,08 Май 23,Ренфилд / Renfield (2023) WEB-DL-HEVC 2160p | ...,18.06 GB,73,94
19,Зарубежные фильмы,08 Май 23,Медуза / Medusa (2021) HDRip-AVC | TVShows,2.12 GB,41,7
20,Зарубежные фильмы,08 Май 23,Медуза / Medusa (2021) BDRip 1080p от New-Team...,11.85 GB,41,17
24,Зарубежные фильмы,07 Май 23,Заклятье. Первое пробуждение / La Niña de la C...,1.37 GB,51,16
25,Зарубежные фильмы,07 Май 23,Стук в хижине / Стук в дверь / Knock at the Ca...,9.97 GB,44,29
30,Зарубежные фильмы,07 Май 23,Банк Дэйва / Bank of Dave (2023) WEB-DLRip-AVC...,1.44 GB,66,13
31,Зарубежные фильмы,07 Май 23,Псих / Siklo (2022) WEBRip 720p | L2,2.69 GB,193,56
69,Зарубежные сериалы,08 Май 23,Список смертников / The Terminal List [S01] (2...,5.91 GB,59,33
70,Зарубежные сериалы,08 Май 23,Медведь / The Bear [S01] (2022) WEB-DLRip | Lo...,2.90 GB,252,70


In [10]:
id = [214]
my_df.loc[id, 'href']

214    http://5.45.84.133/torrent/923581/iron-harvest...
Name: href, dtype: object